In [1]:
# to prepare the Python environment
import numpy as np
import scipy as sp
import scipy.constants as const
import pandas as pd
import os.path
import time
from scipy.optimize import curve_fit
from scipy import interpolate
import warnings
import gc
from numbers import Number

%matplotlib inline

# %reload_ext autoreload
# %autoreload 2

import pyradi.ryplot as ryplot
import pyradi.ryplanck as ryplanck
import pyradi.ryfiles as ryfiles
import pyradi.rymodtran as rymodtran
import pyradi.ryutils as ryutils
import pyradi.rypflux as rypflux

from IPython.display import HTML
from IPython.display import Image
from IPython.display import display
from IPython.display import FileLink, FileLinks

import matplotlib as mpl
mpl.rc("savefig", dpi=300)
mpl.rc('figure', figsize=(10,8))
# %config InlineBackend.figure_format = 'svg'

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 80)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 150)
pd.options.display.float_format = '{:3e}'.format

# suppress the pytables performance warning
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

# Using Pandas

In [5]:
xfilename = 'data/input-parameters.xlsx'

print('Input file      : {}\n  created       : {}\n  last modified : {}\n'.format(xfilename,
                    time.ctime(os.path.getctime(xfilename)),
                    time.ctime(os.path.getmtime(xfilename))))

# read the sensor parameters from the Excel input file
dfSenss = pd.read_excel(xfilename, 'sensors', index_col=None, na_values=['NA'])

# set the first column as the index 
dfSenss.set_index(keys=dfSenss.columns[0],inplace=True)

# get rid of empty lines in excel spreadsheet
dfSenss.dropna(inplace=True)

# where possible convert all to numpy floats
for ccol in dfSenss.columns:
    try:
        dfSenss[ccol] = dfSenss[ccol].astype(np.float64)
    except:
        pass

#calculate some new row data from the info read in from file
# get max spatial frequency for sensor aperture and wavelength
dfSenss.loc['SpatFreqMax cy/m'] = dfSenss.loc['OpticsDia'] / dfSenss.loc['SpecCent']
dfSenss = dfSenss.append(pd.DataFrame(index=['SpatFreq cy/m']),sort=True)

# calculate a spatial freq vector as element in the dataframe (different for each sensor)
for sensor in list(dfSenss.columns):
    dfSenss.loc['SpatFreq cy/m'][sensor] = np.linspace(0, dfSenss.loc['SpatFreqMax cy/m'][sensor], 100)
    
# print, but drop the freq vector for print only
print(dfSenss.drop(['SpatFreq cy/m'], axis=0))


Input file      : data/input-parameters.xlsx
  created       : Mon Nov 28 11:45:52 2016
  last modified : Mon Nov 28 13:16:41 2016

                   MWIR Sensor A   NIR Sensor A   SWIR Sensor A   Vis Sensor A
Band                        MWIR            NIR            SWIR            VIS
Type                         MCT           CMOS             MCT           CMOS
TauOpticsIn         8.000000e-01   7.000000e-01    7.400000e-01   7.000000e-01
TauFilterIn       MWIR-specA.txt  NIR-specA.txt  SWIR-specA.txt  VIS-specA.txt
SpecCent                       4   5.000000e-01               4   5.000000e-01
FrameRate                    100             50             100             50
PixelsH                      640           1280             640           1280
PixelsV                      512            720             512            720
PixelSize           1.500000e-05   5.500000e-06    1.500000e-05   5.500000e-06
FillFactor          9.800000e-01   9.800000e-01    9.800000e-01   9.800000e-01

# Using pyxl to read regions

In [6]:
import openpyxl as pyxl

def readXLSrange(inputfilename,colnames,sheet,cellrange):
    """Read a specified range from a specified sheet in XLS file"""
    
    #load the narcissus electron count vs field angle
    wb = pyxl.load_workbook(inputfilename)
    # print(wb.get_sheet_names())
    ws = wb[sheet]

    #read the table from excel into dataframe, dropping some cols
    table = np.array([[cell.value for cell in col] for col in ws[cellrange]])
    dfTable = pd.DataFrame(table, columns=colnames)
    
    # where possible convert all to numpy floats
    for ccol in dfTable.columns:
        try:
            dfTable[ccol] = dfTable[ccol].astype(np.float64)
        except:
            pass
    return dfTable
        

colnames = [u'Field', u'176.0mm', u'160.0mm', u'142.0mm', u'125.0mm',
       u'107.0mm', u'91.0mm', u'76.0mm', u'62.0mm', u'50.0mm',
       u'40.0mm', u'29.9mm', u'23.7mm']

readXLSrange('data/tableXL.xlsx',colnames=colnames, sheet=u'table',cellrange='U7:AG32')

,Field,176.0mm,160.0mm,142.0mm,125.0mm,107.0mm,91.0mm,76.0mm,62.0mm,50.0mm,40.0mm,29.9mm,23.7mm
0,0.000000e+00,6.135045e-02,6.258473e-02,6.372674e-02,6.533609e-02,6.759124e-02,6.905507e-02,6.924959e-02,6.944369e-02,6.961892e-02,7.005315e-02,7.076776e-02,7.175100e-02
1,4.000000e-02,6.126860e-02,6.234343e-02,6.356214e-02,6.511287e-02,6.733992e-02,6.862622e-02,6.867254e-02,6.859225e-02,6.854954e-02,6.853177e-02,6.889804e-02,6.968222e-02
2,8.000000e-02,6.103787e-02,6.200015e-02,6.337948e-02,6.494735e-02,6.704509e-02,6.818792e-02,6.808632e-02,6.775384e-02,6.736760e-02,6.711221e-02,6.708294e-02,6.735516e-02
3,1.200000e-01,6.074094e-02,6.168701e-02,6.288628e-02,6.460022e-02,6.664008e-02,6.777846e-02,6.736588e-02,6.690680e-02,6.616531e-02,6.573265e-02,6.483450e-02,6.439906e-02
4,1.600000e-01,6.056637e-02,6.136539e-02,6.256564e-02,6.416247e-02,6.626176e-02,6.721909e-02,6.681102e-02,6.603783e-02,6.526547e-02,6.413982e-02,6.266301e-02,6.192084e-02
5,2.000000e-01,6.035191e-02,6.112642e-02,6.215313e-02,6.373097e-02,6.568723e-02,6.665086e-02,6.611921e-02,6.526836e-02,6.403197e-02,6.260929e-02,6.144403e-02,5.895925e-02
6,2.400000e-01,5.985811e-02,6.070108e-02,6.170437e-02,6.316468e-02,6.506137e-02,6.588819e-02,6.528115e-02,6.431760e-02,6.281959e-02,6.151600e-02,6.000232e-02,5.359954e-02
7,2.800000e-01,5.912131e-02,5.986693e-02,6.089282e-02,6.230718e-02,6.388004e-02,6.466169e-02,6.405285e-02,6.292206e-02,6.154999e-02,6.077322e-02,5.308393e-02,5.317748e-02
8,3.200000e-01,5.836705e-02,5.903308e-02,5.990975e-02,6.124094e-02,6.266820e-02,6.339098e-02,6.262894e-02,6.131315e-02,6.020535e-02,5.823463e-02,5.234250e-02,5.259667e-02
9,3.600000e-01,5.753363e-02,5.810667e-02,5.886803e-02,5.998031e-02,6.145960e-02,6.208310e-02,6.116154e-02,5.980377e-02,5.850243e-02,5.496609e-02,5.169327e-02,5.200561e-02
